## Battle of Neighborhood

### Loading Required Libraries

In [1]:
import pandas as pd
import requests
import numpy as np
import requests
import folium

In [2]:
!pip install lxml

### Reading data and creating dataframe

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Cleaning Dataframe

In [4]:
df.drop(df[df['Borough']=="Not assigned"].index, inplace = True) 
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Grouping data in dataframe

In [5]:
df1=df.groupby("Postal Code").agg(lambda x:','.join(set(x)))
df1=df1.reset_index()
df1.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
#If a neighborhood name is empty and borough is non-empty, use the borough name as neighborhood name
df1.loc[df1['Neighborhood'].isnull(),'Neighborhood']=df1.loc[df1['Neighborhood'].isnull(),'Borough']
df1.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Accessing Lat Long values

In [7]:
df2=pd.read_csv('http://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Data Correction & Merging of dataframes

In [8]:
df3= pd.merge(df1, df2)
df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
toronto_data= df3[df3['Borough'].str.contains("Toronto" or "toronto")].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Accessing MAP & plotting

In [10]:
map_toronto = folium.Map(location=[43.6532,-79.3832], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Accessing FOURSQUARE API for additional data requirement

In [14]:
#Foursquare credentials
# Cell code has been removed after storing the foursquare credentials to variables
CLIENT_ID='AUBTNBUQXJTKRUAW4BOMGGOF5WDGWBFKNGY1IQDRWH5M5X5P'
CLIENT_SECRET='EKRAVL4G5GHDKIYCL1FG5U40PTUMBPMWWZD25VQEQCQTWYBC'
VERSION = '20180605'

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
#Getting venues near all the neighbirhoods in Toronto
LIMIT=100
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [17]:
print(toronto_venues.shape)
toronto_venues.head()

(3188, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
3,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
4,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


#### We see that we've acquired 3189 venues near the locations of our dataframe

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,...,0.010000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.01,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,...,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.00,0.00,0.000000,0.000000,0.020408,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.00,0.066667,0.066667,0.000000,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
4,Central Bay Street,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,...,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.01,0.000000,0.020000
5,Christie,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,...,0.020000,0.000000,0.00,0.010000,0.00,0.010000,0.000000,0.00,0.000000,0.000000
6,Church and Wellesley,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.01,0.000000,0.00,0.000000,0.000000,0.01,0.000000,0.020000
7,"Commerce Court, Victoria Hotel",0.00,0.00,0.000000,0.000000,0.020000,0.00,0.00,0.00,0.00,...,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
8,Davisville,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,...,0.010000,0.010000,0.00,0.010000,0.00,0.010000,0.000000,0.00,0.000000,0.010000
9,Davisville North,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,...,0.010000,0.010000,0.00,0.010000,0.00,0.010000,0.000000,0.00,0.000000,0.020000



Now we have 274 unique venue categories, around 39 different neighborhoods of Toronto, and their relative presence in each neighborhood
#### For finding a neighborhood for our Indian Restaurant, we will mainly focus on 2 things for the primary analysis

    1. Neighborhoods with a large number of restaurants clustered around are more likely to be spots having a footfall influx, thus already have a potential market.
    2. We are finding a place where we don't have competition from the exact same cuisine.

#### We will make another dataframe, which contains the data of only Restaurants as category


In [20]:
sample=toronto_grouped.filter(regex='Restaurant')

In [21]:
sample.head()

,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,...,Sushi Restaurant,Syrian Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,0.010000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.01,0.0,...,0.000000,0.0,0.0,0.00,0.010000,0.0,0.00,0.0,0.01,0.0
1,0.010000,0.0,0.0,0.0,0.0,0.0,0.01,0.00,0.01,0.0,...,0.000000,0.0,0.0,0.01,0.000000,0.0,0.03,0.0,0.02,0.0
2,0.020408,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,...,0.040816,0.0,0.0,0.00,0.020408,0.0,0.00,0.0,0.00,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,...,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.00,0.0
4,0.010000,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.00,0.0,...,0.030000,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.02,0.0


In [22]:
sample.shape

(39, 53)

### We've found 53 different kinds of Restaurants

In [23]:
for item in sample.columns:
    print(item)

American Restaurant
Asian Restaurant
Belgian Restaurant
Brazilian Restaurant
Cajun / Creole Restaurant
Cantonese Restaurant
Caribbean Restaurant
Chinese Restaurant
Comfort Food Restaurant
Cuban Restaurant
Doner Restaurant
Dumpling Restaurant
Eastern European Restaurant
Ethiopian Restaurant
Falafel Restaurant
Fast Food Restaurant
Filipino Restaurant
French Restaurant
German Restaurant
Greek Restaurant
Hawaiian Restaurant
Indian Chinese Restaurant
Indian Restaurant
Indonesian Restaurant
Italian Restaurant
Japanese Restaurant
Jewish Restaurant
Korean Restaurant
Latin American Restaurant
Malay Restaurant
Mediterranean Restaurant
Mexican Restaurant
Middle Eastern Restaurant
Modern European Restaurant
New American Restaurant
North Indian Restaurant
Pakistani Restaurant
Persian Restaurant
Portuguese Restaurant
Ramen Restaurant
Restaurant
Seafood Restaurant
South American Restaurant
Sushi Restaurant
Syrian Restaurant
Taiwanese Restaurant
Tapas Restaurant
Thai Restaurant
Theme Restaurant
Tibeta

#### Now, we remove the Indian restaurant column from this dataframe

In [24]:
sample.drop(['Indian Restaurant'], axis = 1,inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [25]:
sample.head()

,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,...,Sushi Restaurant,Syrian Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,0.010000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.01,0.0,...,0.000000,0.0,0.0,0.00,0.010000,0.0,0.00,0.0,0.01,0.0
1,0.010000,0.0,0.0,0.0,0.0,0.0,0.01,0.00,0.01,0.0,...,0.000000,0.0,0.0,0.01,0.000000,0.0,0.03,0.0,0.02,0.0
2,0.020408,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,...,0.040816,0.0,0.0,0.00,0.020408,0.0,0.00,0.0,0.00,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,...,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.00,0.0
4,0.010000,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.00,0.0,...,0.030000,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.02,0.0


#### Now we rejoin the neighborhood column to this dataframe

In [26]:
sample['Neighborhood'] =toronto_grouped['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sample.columns[-1]] + list(sample.columns[:-1])
sample = sample[fixed_columns]

sample.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighborhood,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,...,Sushi Restaurant,Syrian Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Berczy Park,0.010000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.01,...,0.000000,0.0,0.0,0.00,0.010000,0.0,0.00,0.0,0.01,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.010000,0.0,0.0,0.0,0.0,0.0,0.01,0.00,0.01,...,0.000000,0.0,0.0,0.01,0.000000,0.0,0.03,0.0,0.02,0.0
2,"Business reply mail Processing Centre, South C...",0.020408,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,0.040816,0.0,0.0,0.00,0.020408,0.0,0.00,0.0,0.00,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.00,0.0
4,Central Bay Street,0.010000,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.00,...,0.030000,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.02,0.0


### Now we sum the rows horizontally, to get an aggregate value for each Neighborhood, representing the aggregation of all Non Indian Restaurants

In [27]:
aggregate=sample.sum(axis=1)

In [28]:
aggregate.head()

0    0.180000
1    0.250000
2    0.204082
3    0.000000
4    0.240000
dtype: float64

In [29]:
sample2= pd.DataFrame(columns=['Neighborhood','Aggregate Restaurants','Indian Restaurants'])

In [30]:
sample2['Neighborhood']=sample['Neighborhood']
sample2['Aggregate Restaurants']=aggregate
sample2.head()

,Neighborhood,Aggregate Restaurants,Indian Restaurants
0,Berczy Park,0.180000,NaN
1,"Brockton, Parkdale Village, Exhibition Place",0.250000,NaN
2,"Business reply mail Processing Centre, South C...",0.204082,NaN
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,NaN
4,Central Bay Street,0.240000,NaN


In [31]:
sample2

,Neighborhood,Aggregate Restaurants,Indian Restaurants
0,Berczy Park,0.180000,NaN
1,"Brockton, Parkdale Village, Exhibition Place",0.250000,NaN
2,"Business reply mail Processing Centre, South C...",0.204082,NaN
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,NaN
4,Central Bay Street,0.240000,NaN
5,Christie,0.310000,NaN
6,Church and Wellesley,0.230000,NaN
7,"Commerce Court, Victoria Hotel",0.220000,NaN
8,Davisville,0.350000,NaN
9,Davisville North,0.250000,NaN



#### This is the complete list of neighborhoods in Toronto with an aggregated representation of all non Indian restaurants, and just Indian restaurants as separate columns

#### Rows with higher number in the aggregate column shows the neighborhoods with a clustering of restaurants, and rows having no value for Indian restaurants shows neighborhoods with no Indian restaurant

In [34]:
#Sorting the database in decreasing order of the aggregate values
sample2.sort_values(by=['Aggregate Restaurants'], inplace=True, ascending=False)
sample2.head(10)

,Neighborhood,Aggregate Restaurants,Indian Restaurants
8,Davisville,0.350000,NaN
19,"Little Portugal, Trinity",0.320000,NaN
36,"The Danforth West, Riverdale",0.310000,NaN
5,Christie,0.310000,NaN
33,"Summerhill West, Rathnelly, South Hill, Forest...",0.307692,NaN
30,"St. James Town, Cabbagetown",0.297297,NaN
20,"Moore Park, Summerhill East",0.285714,NaN
13,"Garden District, Ryerson",0.270000,NaN
32,Studio District,0.270000,NaN
34,"The Annex, North Midtown, Yorkville",0.270000,NaN



##### From the above data frame, let's consider the top 5 rows, we see that that Davisville has the highest clustering of restaurants around, and it also has the highest number of Indian restaurants around. In fact, all neighborhoods in the top 5, except Little Portugal has an Indian restaurant operating nearby.

##### We can also see that Little Portugal and the Danforth West has the same value for aggregate restaurants, making it a tie for second place, but makes it a better option to start our Indian restaurant. Hence, we can focus more on the Little Portugal neighborhood

##### One important task left before us now is making sure that the neighborhood we have selected is cosmopolitan in nature. We don't want to take a risk of residents of the area not being interested to try out a new cuisine

##### We will now see what other cuisines are available around the Little Portugal region


In [35]:
sample3=sample.loc[sample['Neighborhood'] == "Little Portugal, Trinity"]

In [36]:
sample3

,Neighborhood,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,...,Sushi Restaurant,Syrian Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
19,"Little Portugal, Trinity",0.01,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.0,0.05,0.01


##### Now we try to find the various cuisines of restaurants operating in the Little Portugal Neighborhood, although from the first look, we see that it has American, Asian,Tapas,Thai,Vegan and Vietnamese cuisines, which would already make it a multi cuisine neighborhood, and that should play in favor of the stakeholder

In [37]:
i=0
for column in sample3.columns:
    if sample3.iloc[0,i]!=0:
        print(column)
    i+=1

Neighborhood
American Restaurant
Asian Restaurant
Cuban Restaurant
Dumpling Restaurant
French Restaurant
Greek Restaurant
Italian Restaurant
Japanese Restaurant
Korean Restaurant
Malay Restaurant
New American Restaurant
Restaurant
Seafood Restaurant
Tapas Restaurant
Thai Restaurant
Vegetarian / Vegan Restaurant
Vietnamese Restaurant


##### We see that the neighborhood has all the above cuisines and a great spot to find a wide variety of cuisines from all over the world, this would rule out the stakeholder's doubt whether a new cuisine will be welcome there, and we can select Little Portugal to be a neighborhood of our choice based on this primary analysis

##### Now, let us plot the Little Portugal neighborhood on the map of Toronto


In [38]:
lat=float(toronto_data.loc[toronto_data['Neighborhood']=='Little Portugal, Trinity','Latitude'])
lng=float(toronto_data.loc[toronto_data['Neighborhood']=='Little Portugal, Trinity','Longitude'])

In [39]:
map_new = folium.Map(location=[43.6532,-79.3832], zoom_start=12)

folium.Marker(
    location=[lat,lng],
    popup='LIttle Portugal'
).add_to(map_new)
map_new

##### From the primary analysis, we have chosen Little Portugal as the preferred neighborhood for starting the new Indian Restaurant. The final decision has to be made only after considering other factors like availability of an ideal space for rent, local traffic, and extra amenities